In [ ]:
%autoreload 2
%pylab inline
%load_ext Cython

In [ ]:
from __future__ import absolute_import, print_function

from collections import OrderedDict
import cPickle as pickle
import os
from os.path import abspath, dirname, isfile, join
import sys
import time

os.sys.path.append(dirname(dirname(abspath('__file__'))))
from retro import (powerspace, spherical_volume, extract_photon_info,
                   shift_and_bin, sphbin2cartbin, pol2cart, sph2cart)
import retro.shift_and_bin
import retro.sphbin2cartbin

from genericUtils import timediffstamp
from plotGoodies import removeBorder

In [ ]:
#tables_dir = '/home/justin/src/retro/retro/data/tables'
tables_dir = '/data/icecube/retro_tables/full1000'
geom_fpath = '/home/justin/src/retro/retro/data/geo_array.npy'

In [ ]:
r_max = 400
r_power = 2
n_rbins = 200
n_costhetabins = 40
n_phibins = 80

r_edges = powerspace(0, r_max, n_rbins + 1, r_power)
theta_edges = np.arccos(np.linspace(1, -1, n_costhetabins + 1))

R, THETA = np.meshgrid(r_edges, theta_edges, indexing='ij')
coords = []
exact_vols = []
for ri in range(n_rbins):
    subcoords = []
    sub_exact_vols = []
    for ti in range(int(np.ceil(n_costhetabins / 2.0))):
        rs = R[ri:ri+2, ti:ti+2]
        ts = THETA[ri:ri+2, ti:ti+2]
        bin_corner_coords = zip(rs.flat, ts.flat)
        dcostheta = np.abs(np.diff(np.cos([ts.max(), ts.min()])))
        exact_vol = spherical_volume(rmin=rs.max(), rmax=rs.min(), dcostheta=dcostheta, dphi=np.pi/2)
        sub_exact_vols.append(exact_vol)
    exact_vols.append(sub_exact_vols)
exact_vols = np.array(exact_vols)

In [ ]:
reload(retro.sphbin2cartbin)

x_bw = y_bw = z_bw = 1
x_oversample = y_oversample = z_oversample = 1
antialias_factor = 1

kwargs = dict(
    r_max=r_max, r_power=r_power,
    n_rbins=n_rbins, n_costhetabins=n_costhetabins, n_phibins=n_phibins,
    x_bw=x_bw, y_bw=y_bw, z_bw=z_bw,
    x_oversample=x_oversample, y_oversample=y_oversample, z_oversample=z_oversample,
    antialias_factor=antialias_factor
)

fname = (
    'sphbin2cartbin_first_octant_mapping'
    '_nr{n_rbins:d}_ncostheta{n_costhetabins:d}_nphi{n_phibins:d}'
    '_rmax{r_max:f}_rpwr{r_power}'
    '_xbw{x_bw:f}_ybw{y_bw:f}_zbw{z_bw:f}'
    '_xos{x_oversample:d}_yos{y_oversample:d}_zos{z_oversample:d}'
    '_aa{antialias_factor:d}'
    '.pkl'.format(**kwargs)
)
fpath = join(tables_dir, fname)

if isfile(fpath):
    sys.stdout.write('loading arrays from file...\n')
    sys.stdout.flush()
    
    t0 = time.time()
    dump_data = pickle.load(file(fpath, 'rb'))
    ind_arrays = dump_data['ind_arrays']
    vol_arrays = dump_data['vol_arrays']
    t1 = time.time()
    print('time to load from pickle:', timediffstamp(t1 - t0))
    
else:
    sys.stdout.write('computing arrays...\n')
    sys.stdout.flush()
    
    t0 = time.time()
    ind_arrays, vol_arrays = retro.sphbin2cartbin.sphbin2cartbin(**kwargs)
    t1 = time.time()
    print('time to compute:', timediffstamp(t1 - t0))
    
    dump_data = OrderedDict([
        ('kwargs', kwargs),
        ('ind_arrays', ind_arrays),
        ('vol_arrays', vol_arrays)
    ])
    pickle.dump(dump_data, file(fpath, 'wb'), protocol=pickle.HIGHEST_PROTOCOL)
    t2 = time.time()
    print('time to pickle the results:', timediffstamp(t2 - t1))

print('')
!ls -hl {fpath}

In [ ]:
binned_vol = np.sum([va.sum() for va in vol_arrays])
exact_vol = spherical_volume(rmin=0, rmax=r_max, dcostheta=-1, dphi=np.pi/2)
print('exact vol = %f, binned vol = %f (%e fract error)'
      % (exact_vol, binned_vol, (binned_vol-exact_vol)/exact_vol))

In [ ]:
ind_bin_vols = np.array([va.sum() for va in vol_arrays])
fract_err = ind_bin_vols/exact_vols.flat - 1
abs_fract_err = np.abs(fract_err)
worst_abs_fract_err = np.max(abs_fract_err)
flat_idx = np.where(abs_fract_err == worst_abs_fract_err)[0][0]
r_idx, costheta_idx = divmod(flat_idx, int(np.ceil(n_costhetabins/2)))
print('worst single-bin fract err: %e; r_idx=%d, costheta_idx=%d; binned vol=%e, exact vol=%e'
      % (worst_abs_fract_err, r_idx, costheta_idx, ind_bin_vols[flat_idx], exact_vols[r_idx, costheta_idx]))

In [ ]:
reload(retro.shift_and_bin)

subdet = 'ic'
depth_slice = slice(48, 48+3)
dom_indices = np.array([25, 26, 34, 35, 36, 44, 45])
#depth_slice = slice(48, 48+1)
#dom_indices = np.array([35])
t_slice = [slice(None), slice(None), slice(None)]
xlims = (-500, 500)
ylims = (-600, 600)
zlims = (-600, 600)


nx = int((xlims[1] - xlims[0]) / x_bw)
ny = int((ylims[1] - ylims[0]) / y_bw)
nz = int((zlims[1] - zlims[0]) / z_bw)
shape = (nx, ny, nz)

# Instantiate accumulation arrays
binned_spv = np.zeros((shape), dtype=np.float64)
binned_px_spv = np.zeros((shape), dtype=np.float64)
binned_py_spv = np.zeros((shape), dtype=np.float64)
binned_pz_spv = np.zeros((shape), dtype=np.float64)
binned_one_minus_sp = np.ones((shape), dtype=np.float64)

geom = np.atleast_2d(np.load(geom_fpath)[depth_slice, dom_indices, :])
print('geom:', geom)
print('geom.shape:', geom.shape)
dom_count = geom.shape[0]
depth_indices = range(60)[depth_slice]

t00 = time.time()
for depth_idx, dom_coords in zip(depth_indices, geom):
    print('depth_idx:', depth_idx)
    #print('dom_coords:', dom_coords)
    #for (x,y,z) in dom_coords:
    #    print(x,y,z)
    
    t0 = time.time()
    table_fname = (
        'retro_nevts1000'
        '_{subdet:s}'
        '_DOM{depth_idx:d}'
        '_r_cz_t_angles'
        '.fits'.format(
            subdet=subdet.upper(), depth_idx=depth_idx
        )
    )
    photon_info, bin_edges = extract_photon_info(
        fpath=join(tables_dir, table_fname),
        dom_depth_index=depth_idx
    )
    t1 = time.time()
    print('time to load the retro table:', timediffstamp(t1 - t0))

    sp = photon_info.survival_prob[depth_idx]
    plength = photon_info.length[depth_idx]
    ptheta = photon_info.theta[depth_idx]
    pdeltaphi = photon_info.deltaphi[depth_idx]
    
    plength *= np.cos(pdeltaphi)
    pz = plength * np.cos(ptheta)
    prho = plength * np.sin(ptheta)
    
    t_indep_sp = 1 - np.prod(1 - sp[t_slice], axis=0)
    
    mask = t_indep_sp != 0
    scale = 1 / sp.sum(axis=0)[mask]
    
    t_indep_pz = np.zeros_like(t_indep_sp)
    t_indep_prho = np.zeros_like(t_indep_sp)
    
    t_indep_pz[mask] = (pz[t_slice] * sp[t_slice]).sum(axis=0)[mask] * scale
    t_indep_prho[mask] = (prho[t_slice] * sp[t_slice]).sum(axis=0)[mask] * scale
    
    t2 = time.time()
    print('time to marginalize out time dim, 1 depth:', timediffstamp(t2 - t1))

    retro.shift_and_bin.shift_and_bin(
        ind_arrays,
        vol_arrays,
        dom_coords,
        t_indep_sp,
        t_indep_prho,
        t_indep_pz,
        n_rbins,
        n_costhetabins,
        binned_spv,
        binned_px_spv,
        binned_py_spv,
        binned_pz_spv,
        binned_one_minus_sp,
        nx,
        ny,
        nz,
        xlims[0],
        ylims[0],
        zlims[0],
        x_bw,
        y_bw,
        z_bw,
        x_oversample,
        y_oversample,
        z_oversample
    )
    t3 = time.time()
    print('time to shift and bin:', timediffstamp(t3 - t2))

print('total time to shift and bin:', timediffstamp(t3 - t00))

binned_sp = 1 - binned_one_minus_sp
del binned_one_minus_sp

mask = binned_spv != 0
binned_px_spv[mask] /= binned_spv[mask]
binned_py_spv[mask] /= binned_spv[mask]
binned_pz_spv[mask] /= binned_spv[mask]
del mask

# Rename so as to not mislead
binned_px = binned_px_spv
binned_py = binned_py_spv
binned_pz = binned_pz_spv
del binned_px_spv, binned_py_spv, binned_pz_spv

t4 = time.time()
print('time to normalize histograms:', timediffstamp(t4 - t3))

In [ ]:
mid_x = int(nx/2-1)
mid_y = int(ny/2-1)
mid_z = int(nz/2-1)

#vmin, vmax = binned_sp.min(), binned_sp.max()

fig = plt.figure(1, figsize=(10, 10), dpi=600)
fig.clf()

ax1 = fig.add_subplot(221)
pc1 = ax1.pcolormesh(
    binned_sp[mid_x, :, :].T,
    cmap='viridis', shading='flat',
    #vmin=vmin, vmax=vmax
)

ax2 = fig.add_subplot(222)
pc2 = ax2.pcolormesh(
    binned_sp[:, mid_y, :].T,
    cmap='viridis', shading='flat',
    #vmin=vmin, vmax=vmax
)

ax3 = fig.add_subplot(223)
pc3 = ax3.pcolormesh(
    binned_sp[:, :, mid_z],
    cmap='viridis', shading='flat',
    #vmin=vmin, vmax=vmax
)

ax4 = fig.add_subplot(224)
removeBorder(ax4, left=False, bottom=False)

for pc in [pc3, pc2, pc1]:
    plt.colorbar(pc)
    
for ax in [ax1, ax2, ax3]:
    ax.axis('image')
fig.suptitle('Survival probability planar slices through DOM')
fig.tight_layout(rect=(0, 0, 1, 0.96))
fig.savefig('slices3.png')
pass

In [ ]:
fig = plt.figure(1, figsize=(10, 10), dpi=120)
fig.clf()
ax1 = fig.add_subplot(221)
ax1.pcolor((1 - (1 - binned_sp).prod(axis=0)).T, cmap='viridis')
ax2 = fig.add_subplot(222)
ax2.pcolor((1 - (1 - binned_sp).prod(axis=1)).T, cmap='viridis')
ax3 = fig.add_subplot(223)
ax3.pcolor((1 - (1 - binned_sp).prod(axis=2)), cmap='viridis')

ax4 = fig.add_subplot(224)
removeBorder(ax4, left=False, bottom=False)

for pc in [pc3, pc2, pc1]:
    plt.colorbar(pc)

for ax in [ax1, ax2, ax3]:
    ax.axis('image')
fig.suptitle('Survival probability aggregated along each axis')
fig.tight_layout(rect=(0, 0, 1, 0.96))
fig.savefig('projections3.png')
pass

In [ ]:
%pylab inline
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
print('n_rbins:', n_rbins, 'n_thetabins:', n_thetabins)
print('tot bins:', n_rbins * n_thetabins, 'len(ind_arrays):', len(ind_arrays))
r_idx = 100
theta_idx = 10
n_thetabins_in_quad = int(np.ceil(float(n_thetabins) / 2.0))
flat_idx = theta_idx + n_thetabins_in_quad * r_idx
#flat_idx = theta_idx + r_idx * n_thetabins
#flat_idx =18 
ind_array = ind_arrays[flat_idx]
print('ind_arrays.shape[0]:', ind_array.shape[0])

fig = plt.figure(1, figsize=(10, 10))
fig.clf()
ax = fig.add_subplot(221, projection='3d')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')
ax.set_xlim(0, xlims[1])
ax.set_ylim(0, ylims[1])
ax.set_zlim(0, zlims[1])
ax.grid(True)
#ax.axis('square')
ax.plot((ind_array[:,0] + 1/2/x_oversample)*x_bw,
        (ind_array[:,1] + 1/2/y_oversample)*y_bw,
        (ind_array[:,2] + 1/2/z_oversample)*z_bw,
        alpha=0.5, marker='o', ms=1, c='r', linestyle='none')
pass